In [90]:
sc

<SparkContext master=spark://spkma:7077 appName=PySparkShell>

In [91]:
Compound = sc.textFile("hdfs://nn/tmp/tmu_medicine_table_compound.csv")
Compound.take(10)

['"1","滅瘢痕","以凍淩頻熨之，良。（《千金方》）","夏冰","4"',
 '"2","闢禳時疫","半天河水，飲之。（《醫林集要》","半天河","4"',
 '"3","身體白駁","取樹木孔中水洗之，搗桂末唾和敷之，日再上。（張文仲《備急方》）","半天河","4"',
 '"4","目不得瞑","乃陽氣盛不得入于陰，陰氣虛，故目不得瞑。治法飲以半夏湯，用流水千裏外者八升，揚之萬遍，取其清五升煮之，炊葦薪火，置秫米一升，半夏五合，徐炊令竭爲一升半，去滓飲汁一小杯，日三飲，以知爲度。詳半夏下。（《靈樞經》","流水","4"',
 '"5","汗後奔豚","茯苓桂枝甘草大棗湯，治發汗後臍下悸，欲作奔豚者。茯苓一兩，炙甘草二錢半，桂枝三錢，大棗二枚。以甘瀾水二升，先煮茯苓，納諸藥煮服之，日三。（張仲景《金匱要略》）","流水","4"',
 '"6","服藥過劑煩悶","東流水飲一、二升。（《肘後方》）","流水","4"',
 '"7","九竅出血","酒後熱痢，洗目中膚翳，治人大驚九竅四肢指歧皆出血，以水 面。和朱砂服，令人好顔色，鎮心安神。","井華水","3"',
 '"8","衄血不止","葉氏用新汲水，随左右洗足即止，累用有效","井華水","0"',
 '"9","衄血不止","用冷水 面","井華水","0"',
 '"10","衄血不止","冷水浸紙貼囟上，以熨鬭熨之，立止","井華水","0"']

In [92]:
c_names = Compound.map(lambda x:x.split(",")[1][1:-1].replace(" ","").replace("KT","").split("，"))
c_names.take(10)

[['滅瘢痕'],
 ['闢禳時疫'],
 ['身體白駁'],
 ['目不得瞑'],
 ['汗後奔豚'],
 ['服藥過劑煩悶'],
 ['九竅出血'],
 ['衄血不止'],
 ['衄血不止'],
 ['衄血不止']]

In [93]:
def f(sentences):
    cut_word = []
    for sentence in sentences:
        cut_word.extend([word for word in sentence])
        cut_word.extend([sentence[i:i+2] for i in range(len(sentence)-1)])
    return cut_word
    
word_list = c_names.map(lambda x:f(x))
word_list.take(10)

[['滅', '瘢', '痕', '滅瘢', '瘢痕'],
 ['闢', '禳', '時', '疫', '闢禳', '禳時', '時疫'],
 ['身', '體', '白', '駁', '身體', '體白', '白駁'],
 ['目', '不', '得', '瞑', '目不', '不得', '得瞑'],
 ['汗', '後', '奔', '豚', '汗後', '後奔', '奔豚'],
 ['服', '藥', '過', '劑', '煩', '悶', '服藥', '藥過', '過劑', '劑煩', '煩悶'],
 ['九', '竅', '出', '血', '九竅', '竅出', '出血'],
 ['衄', '血', '不', '止', '衄血', '血不', '不止'],
 ['衄', '血', '不', '止', '衄血', '血不', '不止'],
 ['衄', '血', '不', '止', '衄血', '血不', '不止']]

In [94]:
word_list = word_list.zipWithIndex().map(lambda x: (x[1], x[0]))
word_list.take(10)

[(0, ['滅', '瘢', '痕', '滅瘢', '瘢痕']),
 (1, ['闢', '禳', '時', '疫', '闢禳', '禳時', '時疫']),
 (2, ['身', '體', '白', '駁', '身體', '體白', '白駁']),
 (3, ['目', '不', '得', '瞑', '目不', '不得', '得瞑']),
 (4, ['汗', '後', '奔', '豚', '汗後', '後奔', '奔豚']),
 (5, ['服', '藥', '過', '劑', '煩', '悶', '服藥', '藥過', '過劑', '劑煩', '煩悶']),
 (6, ['九', '竅', '出', '血', '九竅', '竅出', '出血']),
 (7, ['衄', '血', '不', '止', '衄血', '血不', '不止']),
 (8, ['衄', '血', '不', '止', '衄血', '血不', '不止']),
 (9, ['衄', '血', '不', '止', '衄血', '血不', '不止'])]

In [95]:
word_list_df = spark.createDataFrame(word_list, ["id", "words"])
word_list_df

DataFrame[id: bigint, words: array<string>]

In [96]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="words", outputCol="rawfeatures")
model = cv.fit(word_list_df)

result = model.transform(word_list_df)
result.show(truncate=False)

+---+------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+
|id |words                                           |rawfeatures                                                                                                                   |
+---+------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+
|0  |[滅, 瘢, 痕, 滅瘢, 瘢痕]                               |(9895,[639,782,868,1849,3115],[1.0,1.0,1.0,1.0,1.0])                                                                          |
|1  |[闢, 禳, 時, 疫, 闢禳, 禳時, 時疫]                        |(9895,[260,320,678,719,1075,2403,3882],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                                         |
|2  |[身, 體, 白, 駁, 身體, 體白, 白駁]                        |(9895,[25,90,516,1137,1217,1741,9409

In [97]:
# from pyspark.ml.feature import IDF

vocab = model.vocabulary
vocab_broadcast = sc.broadcast(vocab)

# idf = IDF(inputCol="rawfeatures", outputCol="features")
# idfModel = idf.fit(result)
# rescaledData = idfModel.transform(result)

# rescaledData.show(5)
# rescaledData.select('id', 'features').take(5)

In [98]:
from pyspark.ml.clustering import LDA

lda = LDA(featuresCol="rawfeatures",k=15)
ldaModel = lda.fit(result)

In [99]:
topics = ldaModel.topicsMatrix()
topics

DenseMatrix(9895, 15, [2.2714, 2.1367, 0.5771, 4.4938, 1.3279, 1.3662, 6.4112, 1.2485, ..., 0.6189, 0.5431, 1.307, 0.6089, 0.5701, 0.5507, 0.6028, 0.6592], 0)

In [100]:
ldatopics = ldaModel.describeTopics()
ldatopics.show(truncate=True)

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[10, 208, 270, 16...|[0.00256623776694...|
|    1|[1, 6, 40, 48, 75...|[0.01347570388495...|
|    2|[19, 0, 5, 37, 67...|[0.01090064094296...|
|    3|[51, 60, 258, 39,...|[0.00344790728457...|
|    4|[66, 416, 402, 43...|[0.00217650050697...|
|    5|[3, 20, 33, 15, 0...|[0.01326934247847...|
|    6|[2, 30, 59, 96, 1...|[0.01657129810884...|
|    7|[710, 1488, 645, ...|[6.15005267026045...|
|    8|[0, 17, 18, 3, 47...|[0.01446659985760...|
|    9|[5, 8, 9, 4, 12, ...|[0.01684169555826...|
|   10|[13, 128, 106, 12...|[0.00662474498048...|
|   11|[10, 1, 269, 33, ...|[0.00364069907111...|
|   12|[154, 28, 30, 143...|[0.00259942594231...|
|   13|[15, 0, 155, 4, 1...|[0.00530656218249...|
|   14|[2, 1, 31, 6, 39,...|[0.01018735961151...|
+-----+--------------------+--------------------+



In [101]:
transformed = ldaModel.transform(result)
transformed.show(5)

+---+--------------------+--------------------+--------------------+
| id|               words|         rawfeatures|   topicDistribution|
+---+--------------------+--------------------+--------------------+
|  0|   [滅, 瘢, 痕, 滅瘢, 瘢痕]|(9895,[639,782,86...|[0.01024884977186...|
|  1|[闢, 禳, 時, 疫, 闢禳, ...|(9895,[260,320,67...|[0.88818087143642...|
|  2|[身, 體, 白, 駁, 身體, ...|(9895,[25,90,516,...|[0.00767049924228...|
|  3|[目, 不, 得, 瞑, 目不, ...|(9895,[0,28,191,3...|[0.00767049848759...|
|  4|[汗, 後, 奔, 豚, 汗後, ...|(9895,[45,131,849...|[0.00767050266801...|
+---+--------------------+--------------------+--------------------+
only showing top 5 rows



In [102]:
topics_rdd = ldatopics.rdd

topics_words = topics_rdd\
    .map(lambda row: row['termIndices'])\
    .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
    .collect()

for idx, topic in enumerate(topics_words):
    print('topic {}\n{}\n'.format(idx, (topic)))

topic 0
['氣', '疝', '疝氣', '人', '聾', '黑', '令', '腫', '發', '耳聾']

topic 1
['痛', '腫', '腫痛', '喉', '痹', '氣', '不', '食', '喉痹', '癰']

topic 2
['便', '不', '小', '小便', '淋', '白', '通', '不通', '傷', '便不']

topic 3
['耳', '入', '入耳', '蟲', '蟲入', '乳', '百蟲', '百', '蚰', '癰']

topic 4
['骨', '蒸', '衣', '衣不', '骨蒸', '不', '胞', '胞衣', '損', '不下']

topic 5
['血', '出', '心', '中', '不', '吐', '水', '腫', '痛', '亂']

topic 6
['瘡', '生', '陰', '癢', '癬', '惡瘡', '惡', '生瘡', '風', '濕']

topic 7
['宣', '宣露', '露', '齒', '腫', '疳宣', '霞杯', '杯', '龈', '紫霞']

topic 8
['不', '止', '不止', '血', '婦', '人', '婦人', '血不', '生', '産']

topic 9
['小', '兒', '小兒', '風', '熱', '下', '毒', '痛', '赤', '痢']

topic 10
['傷', '蛇', '咬', '螫', '毒', '犬', '犬咬', '咬傷', '人', '螫傷']

topic 11
['氣', '痛', '氣痛', '心', '核', '結', '結核', '男', '瘰', '不']

topic 12
['翳', '目', '生', '痘', '障', '疹', '目生', '目中', '痰', '頭痛']

topic 13
['中', '不', '脫', '風', '肛', '脫肛', '子', '中風', '不瘥', '腹']

topic 14
['瘡', '痛', '牙', '腫', '蟲', '毒', '齒', '疔', '目', '不']

